In [55]:
import openai
import os
from dotenv import load_dotenv

from langchain.document_loaders import PyMuPDFLoader

#### OPEN AI Settings

In [56]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [102]:
def get_completion(
        prompt: str,
        system_prompt='',
        prev_messages=[]
    ):
    messages = [{"role": "system", "content": system_prompt}]
    messages.extend(prev_messages)
    messages.append({"role": "user", "content": prompt.strip()})
    # For CLI info
    print("--Sending request to OpenAI endpoint--")

    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo-16k',
            messages=messages,
            temperature=0,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=2,
            max_tokens = 400
        )
    except (openai.error.APIError, openai.error.RateLimitError, openai.error.APIConnectionError) as e:
        # Handle API errors and retry the request
        print("API Error:", e)
        print("Retrying the request...")
        return get_completion(prompt, system_prompt, prev_messages)
    
    return response.choices[0].message["content"]

#### Loading PDF

In [44]:
loader = PyMuPDFLoader('doc.pdf').load()
all_text = []
for i in range(len(loader)):
    text = loader[i].page_content
    all_text.append(text)

all_text = ','.join(all_text)

## Method 1: Passing all Context in Prompt

In [58]:
categories = ['Social Advertising', 'Search Advertising', 'Digital Display Advertising', 'Digital Video Advertising', 
 'Partner/Affiliate/Co-OP on Digital Channels', 'Digital Audio Advertising', 'Digital Out of Home', 'SEO',
 'Email Marketing', 'Content and Messaging', 'Organic Social/Influencer', 'SMS/In App Messaging']

In [107]:
query = """
    Read the information and trends from the context and assign a marketing budget to the given categories.
    OUTPUT FORMAT: ONLY JSON with categories as keys and assigned budget in percentages as values.
    Also mention the reasons for assigning percentages to methods.
"""

In [111]:
prompt = f"""
Context: "{all_text}"
Categories: "{categories}"
{query}
"""

In [109]:
result = get_completion(prompt)

--Sending request to OpenAI endpoint--


In [110]:
print(result)

{
  "Social Advertising": 20,
  "Search Advertising": 15,
  "Digital Display Advertising": 10,
  "Digital Video Advertising": 15,
  "Partner/Affiliate/Co-OP on Digital Channels": 5,
  "Digital Audio Advertising": 5,
  "Digital Out of Home": 5,
  "SEO": 10,
  "Email Marketing": 10,
  "Content and Messaging": 5,
  "Organic Social/Influencer": 5,
  "SMS/In App Messaging": 5
}

Reasoning:
1. Social Advertising: Given the popularity and reach of social media platforms, allocating a higher budget to social advertising will help in reaching a wider audience and driving engagement.
2. Search Advertising: Investing in search advertising is important as it helps in capturing potential customers who are actively searching for products or services related to your business.
3. Digital Display Advertising: Allocating a moderate budget to digital display advertising can help in increasing brand visibility and reaching a broader audience across various websites and apps.
4. Digital Video Advertising: 

## Method 2: Without Context/Document

In [94]:
categories = ['Social Advertising', 'Search Advertising', 'Digital Display Advertising', 'Digital Video Advertising', 
 'Partner/Affiliate/Co-OP on Digital Channels', 'Digital Audio Advertising', 'Digital Out of Home', 'SEO',
 'Email Marketing', 'Content and Messaging', 'Organic Social/Influencer', 'SMS/In App Messaging']

In [98]:
query = """
    Act as a Marketing Manager and devise a marketing campaign strategy. Assign a marketing budget to the given categories.
    OUTPUT FORMAT: ONLY JSON with categories as keys and assigned budget in percentages as values.
    Also mention the reasons for assigning percentages to methods.
"""

In [99]:
prompt = f"""
Categories: "{categories}"
{query}
"""

In [104]:
result = get_completion(prompt)

--Sending request to OpenAI endpoint--


In [106]:
print(result)

{
  "Social Advertising": 20,
  "Search Advertising": 15,
  "Digital Display Advertising": 10,
  "Digital Video Advertising": 15,
  "Partner/Affiliate/Co-OP on Digital Channels": 5,
  "Digital Audio Advertising": 5,
  "Digital Out of Home": 5,
  "SEO": 10,
  "Email Marketing": 10,
  "Content and Messaging": 5,
  "Organic Social/Influencer": 5,
  "SMS/In App Messaging": 5
}

Reasons for assigning percentages to methods:

1. Social Advertising (20%): Social media platforms have a wide reach and offer targeted advertising options, making it an effective channel for reaching the target audience.

2. Search Advertising (15%): Search engine ads can capture users actively searching for products or services, providing high intent leads. Allocating a significant portion ensures visibility in search results.

3. Digital Display Advertising (10%): Display ads help increase brand awareness and reach a broader audience. While not as targeted as other channels, they contribute to overall campaign vi

## Method 3: Getting context from Similarity Search

In [121]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = PyMuPDFLoader('doc.pdf').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [136]:
categories = ['Social Advertising', 'Search Advertising', 'Digital Display Advertising', 'Digital Video Advertising', 
 'Partner/Affiliate/Co-OP on Digital Channels', 'Digital Audio Advertising', 'Digital Out of Home', 'SEO',
 'Email Marketing', 'Content and Messaging', 'Organic Social/Influencer', 'SMS/In App Messaging']

In [141]:
context = []

for i in categories:
    query = f"How much should be spent on {i} for marketing campaign?"
    docs = db.similarity_search(query)
    context.append(docs[0].page_content)
    context.append(docs[1].page_content)
    # print(f"\n* {i} *")
    # print(docs[0].page_content)
    # print("\n------------------------\n")



* Social Advertising *
year, 24% of marketers expect to have to reduce their marketing budget, 20% expect to have to adjust their marketing 
goals, 20% believe they’d have to reduce their scope, and 19% report they’d have to pivot their tactics completely. 
On the other hand, the channels they’d expect to keep 
running even in the face of an economic downturn  
are those that generate the strongest returns.
• Paid social media content
• Print ads 
• In-person events 
• Content — blog and SEO
• Podcasts

------------------------


* Search Advertising *
17
State of Marketing Trends 2023
Contacting customers and prospects via SMS, 
Facebook messenger, and WhatsApp is also  
growing in popularity. With more and more 
customers shopping from their phone, delivering 
ads, promotions, and discounts via text or 
messenger increases the chance that you’re 
delivering the right message at the right time.  
In fact, automated marketing campaign workflows 
with SMS can increase conversions by 21

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-32zXsaqWgFJatmuBZbdQhfIc on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-32zXsaqWgFJatmuBZbdQhfIc on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/


* Digital Video Advertising *
34
State of Marketing Trends 2023
When it comes to spending, brands revealed that they upped their 
investment in video in 2022. Compared to 2021, more businesses 
allocated at least 10% of their overall marketing budget to video. 
What are brands planning for 2023? Even with economists predicting a likely recession ahead, only 3% of 
companies plan to reduce their video budget. One in seven companies plan to make a heavier investment 
in video, upping their video spend by more than 10%.

------------------------



Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-32zXsaqWgFJatmuBZbdQhfIc on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-32zXsaqWgFJatmuBZbdQhfIc on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/


* Partner/Affiliate/Co-OP on Digital Channels *
year, 24% of marketers expect to have to reduce their marketing budget, 20% expect to have to adjust their marketing 
goals, 20% believe they’d have to reduce their scope, and 19% report they’d have to pivot their tactics completely. 
On the other hand, the channels they’d expect to keep 
running even in the face of an economic downturn  
are those that generate the strongest returns.
• Paid social media content
• Print ads 
• In-person events 
• Content — blog and SEO
• Podcasts

------------------------



...


* Content and Messaging *
17
State of Marketing Trends 2023
Contacting customers and prospects via SMS, 
Facebook messenger, and WhatsApp is also  
growing in popularity. With more and more 
customers shopping from their phone, delivering 
ads, promotions, and discounts via text or 
messenger increases the chance that you’re 
delivering the right message at the right time.  
In fact, automated marketing campaign workflows 
with SMS can increase conversions by 21%.
Twenty-nine percent of marketers use a search-optimized 
website and blog to attract and convert leads. SEO work is 
a long game, so some marketing teams looking for quicker 
wins sometimes overlook how powerful it can be. But once 
a company sets up a blog and optimizes its webpages for 
search, it lays a foundation to significantly increase web 
traffic and inbound leads.
Though it’s time-consuming, SEO work requires a relatively 
low financial investment, especially when compared to 
strategies like paid ads, events, and 

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-32zXsaqWgFJatmuBZbdQhfIc on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-32zXsaqWgFJatmuBZbdQhfIc on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/


* Organic Social/Influencer *
of Gen Zers have bought a product based on an 
influencer’s recommendation in the past three 
months. More than one in four marketers currently 
leverage influencer marketing, and it offers the 
second-highest ROI of any trend.
Of the companies that worked with influencers last year, the 
influencers they worked with had varying audience sizes.
Nano-influencers,  
with 1,000-9,999 followers or subscribers
Micro-influencers,  
with 10,000-99,999 followers or subscribers
Macro-influencers,

------------------------



Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-32zXsaqWgFJatmuBZbdQhfIc on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-32zXsaqWgFJatmuBZbdQhfIc on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/


* SMS/In App Messaging *
17
State of Marketing Trends 2023
Contacting customers and prospects via SMS, 
Facebook messenger, and WhatsApp is also  
growing in popularity. With more and more 
customers shopping from their phone, delivering 
ads, promotions, and discounts via text or 
messenger increases the chance that you’re 
delivering the right message at the right time.  
In fact, automated marketing campaign workflows 
with SMS can increase conversions by 21%.
Twenty-nine percent of marketers use a search-optimized

------------------------



In [143]:
print(len(context))

24


In [144]:
text_context = ','.join(context)

In [147]:
query = """
    Read the information and trends from the context and assign a marketing budget to the given categories.
    OUTPUT FORMAT: ONLY JSON with categories as keys and assigned budget in percentages as values.
    Also mention the reasons for assigning percentages to methods.
"""

In [148]:
prompt = f"""
Context: "{text_context}"
Categories: "{categories}"
{query}
"""

In [150]:
result = get_completion(prompt)

--Sending request to OpenAI endpoint--


In [151]:
print(result)

{
  "Social Advertising": 15,
  "Search Advertising": 10,
  "Digital Display Advertising": 8,
  "Digital Video Advertising": 12,
  "Partner/Affiliate/Co-OP on Digital Channels": 5,
  "Digital Audio Advertising": 3,
  "Digital Out of Home": 2,
  "SEO": 20,
  "Email Marketing": 10,
  "Content and Messaging": 7,
  "Organic Social/Influencer": 8,
  "SMS/In App Messaging": 0
}

Reasons for assigning percentages:
1. Social Advertising: Given the popularity and effectiveness of paid social media content, allocating a higher budget percentage can help reach a wider audience and generate strong returns.
2. Search Advertising: With 29% of marketers using search-optimized websites and blogs to attract leads, investing in search advertising can complement SEO efforts and drive targeted traffic.
3. Digital Display Advertising: Allocating a moderate budget percentage to digital display advertising allows for brand visibility and reaching audiences across various online platforms.
4. Digital Video Ad